In [1]:
import pandas as pd

In [15]:
df = pd.read_csv('./data/soratena_tonga.csv',date_parser="yy-mm-dd hh:mm:ss")
df.head()

,SoratenaID,DATE(UTC),ARPRSS,SoratenaName,LATD,LOND
0,1001,2022-01-15 11:00:00,997.7,鶴居タンチョウパパ,43.226,144.309
1,1001,2022-01-15 11:01:00,997.7,鶴居タンチョウパパ,43.226,144.309
2,1001,2022-01-15 11:02:00,997.8,鶴居タンチョウパパ,43.226,144.309
3,1001,2022-01-15 11:03:00,997.8,鶴居タンチョウパパ,43.226,144.309
4,1001,2022-01-15 11:04:00,997.9,鶴居タンチョウパパ,43.226,144.309


In [18]:
type(df["DATE(UTC)"][0])

str

In [4]:
df.describe()

,SoratenaID,ARPRSS,LATD,LOND
count,764935.000000,764935.000000,764935.000000,764935.000000
mean,21404.936413,1000.603509,35.887148,136.873705
std,13172.020942,27.311418,2.783114,3.646831
min,1001.000000,828.700000,24.330000,123.819000
25%,11073.000000,995.400000,34.435000,134.238000
50%,21056.000000,1010.500000,35.383000,137.189000
75%,32014.000000,1016.700000,36.539000,139.798000
max,47039.000000,1032.500000,45.448000,145.582000


In [5]:
df.shape

(764935, 6)

In [10]:
df["SoratenaID"].unique().shape

(1583,)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 764935 entries, 0 to 764934
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   SoratenaID    764935 non-null  int64  
 1   DATE(UTC)     764935 non-null  object 
 2   ARPRSS        764935 non-null  float64
 3   SoratenaName  764935 non-null  object 
 4   LATD          764935 non-null  float64
 5   LOND          764935 non-null  float64
dtypes: float64(3), int64(1), object(2)
memory usage: 35.0+ MB
